### Convert the excel files into publication-author-long data

### Import The Packages

In [ ]:
import pandas as pd
import numpy as np
import os
import re

### A function that performs the conversion

In [ ]:
def long_data(file):
    # takes a dataframe as input 

    # transpose the data
    col = file.T

    # change column names to the first row
    col.columns = col.iloc[0]
    col = col.drop('Unnamed: 0')

    vars = []
    vars1 = []
    vars2 = []
    vars3 = []
    vars4 = []

    # generate lists of variables for different purposes
    for var in ['authors__author__@auid__', 'authors__author__affiliation__', 'authors__author__preferred-name__']:
        for each in file['Unnamed: 0']:
            if var in each:
                vars.append(each)

    for var in ['@id__','affiliation-city__', 'affiliation-country__', 'affilname__']:
        for each in file['Unnamed: 0']:
            if each.startswith(var):
                vars.append(each)
                vars1.append(each)

    for var in ['affiliation__@id','affiliation__affiliation-city__', 'affiliation__affiliation-country__', 'affiliation__affilname']:
        for each in file['Unnamed: 0']:
            if each.startswith(var):
                vars.append(each)
                vars2.append(each)

    for var in ['subject-areas__subject-area__$']:
        for each in file['Unnamed: 0']:
            if each.startswith(var):
                vars.append(each)
                vars3.append(each)

    for var in ['subject-areas__subject-area__@code__']:
        for each in file['Unnamed: 0']:
            if each.startswith(var):
                vars.append(each)
                vars4.append(each)

    dic = {}
    count = 0
    for each in col.index:
        #print(each)
        x = col.loc[each].dropna(axis=0)
        num = 0

        # find the number of variables without null values
        for index in x.index:
            if 'authors__author__@auid__' in index:
                num +=1

        # year of publication
        year = col.at[each, 'item__bibrecord__head__source__publicationdate__year__']

        # eid for publication
        eid = col.at[each, 'coredata__eid__']

        # number of citations of the publication
        cite_count = col.at[each, 'coredata__citedby-count__']

        # publication's abstract
        pub_abs = col.at[each, 'item__bibrecord__head__abstracts__']

        # title of publication
        pub_title = col.at[each, 'coredata__dc:title__']

        # whether author is focal author
        focal_author = col.at[each, 'auid'][:-5]
        
        # journal name
        try:
            journal = col.at[each, 'item__bibrecord__head__source__sourcetitle__']
        except: 
            try:
                journal = col.at[each, 'item__bibrecord__head__source__sourcetitle__$__']
            except:
                journal = ''

        # subject area of the publication
        sub_area = [col.at[each, v] for v in vars3 if not(str(col.at[each, v])=='nan')]
        sub_code = [col.at[each, v] for v in vars4 if not(str(col.at[each, v])=='nan')]

        for i in np.arange(0,num):
            
            # author id
            id = col.at[each,'authors__author__@auid__'+str(i)+'__']
            f = focal_author == id
            flag = 0

            if 'authors__author__affiliation__' + str(i) + '__@id__' in vars:
                #print('authors__author__affiliation__' + str(i) + '__@id__')
                affil_id0 = col.at[each, 'authors__author__affiliation__' + str(i) + '__@id__']
                if str(affil_id0) == 'nan':
                    flag = 1
            else:
                flag = 1
            
            if flag == 1:
                ct = 0
                for var in vars:
                    # if the variable name matches the following name format
                    matched = bool(re.match("authors__author__affiliation__"+str(i)+"__@id__"+"[0-9]+", var))
                    if matched:
                        ct+=1

            # parts of author's name
            gname = col.at[each, 'authors__author__preferred-name__' + str(i) + '__ce:given-name__']
            iname = col.at[each, 'authors__author__preferred-name__' + str(i) + '__ce:indexed-name__']
            sname = col.at[each, 'authors__author__preferred-name__' + str(i) + '__ce:surname__']

            # author's ffiliation name, city, country
            if flag == 0:
                name0 = ''
                city0 = ''
                country0 = ''
                if (len(vars2)==0):
                    for j in np.arange(0,int(len(vars1)/4)):
                        if (col.at[each,'@id__'+str(j)+'__'] == affil_id0):
                            name0 = col.at[each, 'affilname__' + str(j)+ '__']
                            city0 = col.at[each, 'affiliation-city__' + str(j) + '__']
                            country0 = col.at[each, 'affiliation-country__' + str(j) + '__']
                            break

                elif (pd.isnull(col.at[each, vars2[0]])):
                    for j in np.arange(0,int(len(vars1)/4)):
                        if (col.at[each,'@id__'+str(j)+'__'] == affil_id0):
                            name0 = col.at[each, 'affilname__' + str(j)+ '__']
                            city0 = col.at[each, 'affiliation-city__' + str(j) + '__']
                            country0 = col.at[each, 'affiliation-country__' + str(j) + '__']
                            break

                else:
                    city0 = col.at[each, vars2[1]]
                    country0 = col.at[each, vars2[2]]
                    name0 = col.at[each, vars2[3]]

                # add to a dictionary
                dic[count]={'eid':eid, 'year': year, 'auid':id, 'given-name':gname, 'index-name':iname, 'surname':sname, 'order-pub':i+1, 'focal author': f, 'affil-id': affil_id0, 'affil-name':name0, 'affil-city':city0, 'affil-country':country0, '# citations': cite_count,'title':pub_title, 'journal': journal,'abstract':pub_abs, 'sub_area': sub_area,'aub_code': sub_code }
                count+=1

            else :
                #dic[count]={'eid':eid, 'year': year, 'auid':id, 'given-name':gname, 'index-name':iname, 'surname':sname, 'order-pub':i+1, 'focal author': f, '# citations': cite_count,'title':pub_title, 'journal': journal,'abstract':pub_abs, 'sub_area': sub_area,'aub_code': sub_code }

                 # author's ffiliation name, city, country
                for p in np.arange(ct):
                    affil = col.at[each, 'authors__author__affiliation__' + str(i) + '__@id__'+str(p)+'__']
                    name = ''
                    city = ''
                    country = ''
                    for j in np.arange(0,int(len(vars1)/4)):
                        if (col.at[each,'@id__'+str(j)+'__'] == affil):
                            name = col.at[each, 'affilname__' + str(j)+ '__']
                            city = col.at[each, 'affiliation-city__' + str(j) + '__']
                            country = col.at[each, 'affiliation-country__' + str(j) + '__']
                            break
                        
                    # add to a dictionary
                    dic[count]={'eid':eid, 'year': year, 'auid':id, 'given-name': gname, 'index-name':iname, 'surname':sname, 'order-pub':i+1, 'focal author': f, 'affil-id': affil, 'affil-name':name, 'affil-city':city, 'affil-country':country, '# citations': cite_count,'title':pub_title, 'journal': journal,'abstract':pub_abs, 'sub_area': sub_area,'aub_code': sub_code }
                    count+=1
                    #dic[count]['affil-id_'+str(p)] = affil
                    #dic[count]['affil-name_'+str(p)] = name
                    #dic[count]['affil-city_'+str(p)] = city
                    #dic[count]['affil-country_'+str(p)] = country

            #count+=1

    return dic

In [ ]:
# list of all author ids
files = os.listdir('C:/Users/nikhi/Dropbox/Migration and Science/shorter_excel')

# list of files that have been converted into publication-author-affiliation long form
files_done = [x for x in os.listdir('C:/Users/nikhi/Dropbox/Migration and Science/long data/') if 'auids_done_' in x]

auids_done = []
for i in np.arange(len(files_done)):
    a = pd.read_excel('C:/Users/nikhi\Dropbox\Migration and Science/long data/auids_done_' + str(i+1) +'.xlsx').T
    a.columns = a.iloc[0]
    auth = a['authors']
    auids_done.extend(auth)

final = pd.DataFrame()
names1 = [] # list of all files being converted in current iteration
names2 = [] # list of files converted till now

#pd.DataFrame(columns = ['eid', 'auid', 'given-name', 'index-name', 'surname',  'order-pub','focal author', 'affil-id_0',  'affil-name_0', 'affil-city_0', 'affil-country_0',  'affil-id_1',  'affil-name_1', 'affil-city_1', 'affil-country_1', '# citations','title', 'journal','abstract','sub_area','aub_code'])
# number of author ids already converted
cnt = len(files_done)
for file in files:
    if (('.json' in file) & (file not in auids_done)):
        # if file not already converted
        names1.append(file)

        # dataframe containing the tractable excel file
        df = pd.read_excel('C:/Users/nikhi\Dropbox\Migration and Science\shorter_excel/' + file)

        # converted to the required publication-author-affiliation long form data
        x = long_data(df)
        df_data = pd.DataFrame.from_dict(x).T
        #df_data.to_excel('long_'+str(l)+'.xlsx', engine='xlsxwriter')

        # append to the dataframe that will be converted to excel
        final = final.append(df_data)
        if len(final) > 500000:
            cnt += 1
            final.to_excel('long_'+str(cnt)+'.xlsx', engine='xlsxwriter')
            final = pd.DataFrame()
            #pd.DataFrame(columns = ['eid', 'auid', 'given-name', 'index-name', 'surname',  'order-pub','focal author', 'affil-id_0',  'affil-name_0', 'affil-city_0', 'affil-country_0',  'affil-id_1',  'affil-name_1', 'affil-city_1', 'affil-country_1', '# citations','title', 'journal','abstract','sub_area','aub_code'])
            
            # list of authors whose data has been converted into the required publication-author-affiliation long form data
            au_dic = {'authors': names1}
            au_df = pd.DataFrame.from_dict(au_dic).T
            au_df.to_excel('auids_done_'+str(cnt)+'.xlsx', engine='xlsxwriter')
            names2.extend(names1)
            names1 = []